In [1]:
import pandas as pd
import os
from scipy import stats
from pyecharts.charts import *
from pyecharts.options import global_options
from pyecharts import options as opts
import numpy as np
import pyecharts
print(pyecharts.__version__)


1.8.1


In [2]:
root=os.path.abspath("../..")

In [3]:
#以下模块用于处理焦虑症发病率随年龄的变化趋势
df_age = pd.read_csv(root+'\\data\\prevalence\\anxiety-disorders-prevalence-by-age.csv')
count=0
for i in df_age.columns[3:16]:
    count+=1
    df_age.rename(columns={i:'p'+str(count)},
                  inplace=True)
df_g=df_age.drop('Code',axis=1).groupby('Entity',as_index=False).mean()
dfAge=df_g[(df_g['Entity']=='China') | (df_g['Entity']=='United States')]
dfAge.set_index('Entity',inplace=True)
dfAge.drop(columns=['Year','Anxiety disorders (share of population) - Sex: Both - Age: All ages',
                    'Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized'],
           axis=1,
           inplace=True)
formater="{0:.02f}".format

dfAge = dfAge.applymap(formater)
dfAge


C:\Users\14253\AppData\Local\Temp\ipykernel_19756\2461479789.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfAge.drop(columns=['Year','Anxiety disorders (share of population) - Sex: Both - Age: All ages',


,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13
Entity,,,,,,,,,,,,,
China,3.25,4.81,3.81,3.32,3.25,3.36,3.56,3.72,3.87,4.01,4.16,4.32,4.64
United States,2.26,5.41,7.16,8.16,8.66,8.91,8.94,8.59,7.96,7.19,6.40,5.85,4.94


In [4]:
x=['5-14','15-19','19-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70+']
y1=dfAge.values[0].tolist()
y2=dfAge.values[1].tolist()

bar = Bar()
 
bar.set_global_opts(title_opts=global_options.TitleOpts(title="各年龄段焦虑发病率"))
bar.add_xaxis(x)
bar.add_yaxis("China", y1, stack="stack1")
bar.add_yaxis("United States", y2, stack="stack2")
 
try:
    bar.render(root+"\\result\\age.html")
except:
    os.mkdir(root+"\\result")
    bar.render(root+"\\result\\age.html")


d:\anaconda3\Lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [5]:
#以下模块用于分析性别因素

df_gender=pd.read_csv(root+'\\data\\prevalence\\anxiety-disorders-prevalence-males-vs-females.csv')
df_gender.rename(columns={'Anxiety disorders (share of population) - Sex: Male - Age: All ages':'Male', 
                          'Anxiety disorders (share of population) - Sex: Female - Age: Age-standardized':'Female'},
                 inplace=True)
df_gender.head()

df_gender=df_gender.drop(columns=['Year','Population (historical estimates)','Continent','Code'],
                         axis=1)
df_gender=df_gender.groupby('Entity',
                            as_index=False).mean()
dfGen=df_gender[(df_gender['Entity']=='China') | (df_gender['Entity']=='United States')]
dfGen.set_index('Entity',
                inplace=True)

dfGen = dfGen.applymap(formater)#formater 的定义在上方
dfGen

,Male,Female
Entity,,
China,2.61,4.24
United States,4.43,7.28


In [6]:
y1=dfGen['Male'].tolist()
y2=dfGen["Female"].tolist()
c = (
    Scatter()
    .add_xaxis(['China',"America"])
    .add_yaxis("Female", y2)
    .add_yaxis("Male", y1)
    
    .set_global_opts(
        title_opts=opts.TitleOpts(title="焦虑症患病率与性别的关系"),
        xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
        yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
        visualmap_opts=opts.VisualMapOpts(
            type_="size",  #映射大小
            max_=8,
            min_=2,
            pos_bottom = 50,pos_right = 0)
    )
    .render(root+"\\result\\gender.html")
)

d:\anaconda3\Lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [7]:
#以下模块用于处理焦虑患病率和国家gdp关系
df_gdp=pd.read_csv(root+'\\data\\prevalence\\anxiety-disorders-prevalence-vs-gdp.csv')
df_gdp.rename(columns={'Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized':'Anxiety',
                       'GDP per capita, PPP (constant 2017 international $)':'GDP',
                       },inplace=True)
df_gdp.drop(columns=['Code','Population (historical estimates)','Continent'],inplace=True)
df_gdp.dropna(inplace=True)
df_gdp=df_gdp[df_gdp["Year"]==2019].drop(columns=["Year"])
df_gdp.head()

,Entity,Anxiety,GDP
30,Afghanistan,4.851035,2079.9219
655,Albania,3.837537,13653.2490
915,Algeria,4.771976,11627.2800
1578,Angola,3.934095,6602.2690
1963,Antigua and Barbuda,4.334173,23638.6860


In [8]:
#地图制作
x=df_gdp['Entity']
y=df_gdp['Anxiety']
value = list(y)
attr = list(x)
 
data = []
for index in range(len(attr)):
    city_ionfo=[attr[index],value[index]]
    data.append(city_ionfo)
 
map = (
    Map()
    .add("焦虑指数",data, "world")
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="世界热力地图"),
        visualmap_opts=opts.VisualMapOpts(max_=df_gdp['Anxiety'].max(), 
                                          min_=df_gdp['Anxiety'].min()
                                          ),
    )
    .render(root+'\\result\\map.html')
)

d:\anaconda3\Lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [9]:
#数学检验
stats.ttest_ind(df_gdp['GDP'],df_gdp['Anxiety'],equal_var=False)

TtestResult(statistic=13.801697241603362, pvalue=1.6474866516880435e-30, df=191.00000102362645)

In [10]:
# 数据准备
x = df_gdp['GDP'].tolist()
y = df_gdp['Anxiety'].tolist()

# 创建散点图
scatter = (
    Scatter()
    .add_xaxis(x)
    .add_yaxis(
        series_name="焦虑指数",
        y_axis=y,
        symbol_size=20,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="焦虑指数与GDP的关系"),
    )
    .render(root+"\\result\\gdp.html")
)


d:\anaconda3\Lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
